# Feature construction by KMeans

Cluster businesses, and cluster users. For each review, add the two cluster labels. 

The idea is that similar users will have similar rating habits, and similar businesses will receive similar ratings from users.

# Code

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.get_data import *
from IPython.display import clear_output
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import Ridge
from src.get_data import *
import ast
import time

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Fetch data, do clustering, and add feature(s)

In [33]:
b_data_old, u_data_old, reviews = get_training_data(b_cols='maybe')

b_data, u_data = add_GMM_features(b_data_old, u_data_old, 4, 8)

(12058, 57)
(12058, 58)
(12058, 59)
(12058, 60)
(12058, 61)


# Now construct design matrix

In [34]:
X_train, y_train = construct_design_matrix(
    b_data, u_data, reviews, return_df=True, verbose=True
)
X_valid, y_valid = construct_design_matrix(
    b_data, u_data, valid_queries, return_df=True, verbose=True
)

Constructing design matrix now.
0/150232 done


KeyboardInterrupt: 

# Regression (ridge)

In [25]:
def rmse(y1, y2):
    return np.sqrt(np.mean((y1-y2)**2))

clf = Ridge(alpha=0.1)
clf.fit(X_train.values, y_train)

y_pred_train = clf.predict(X_train.values)
ypt_err = rmse(y_train, y_pred_train)

y_pred_valid = clf.predict(X_valid.values)
ypv_err = rmse(y_valid, y_pred_valid)

print('%f  <- training RMSE' % ypt_err)
print('%f  <- validation RMSE' % ypv_err)

1.004011  <- training RMSE
1.052490  <- validation RMSE
